# Command interface

DSC2 implements 2 command programs, `dsc` and `dsc-query`, for executing DSC and extracting result from executed benchmarks, respectively.

## DSC main program

In [1]:
! dsc -h

usage: dsc [-h] [--version] [-o str] [--target str [str ...]] [--skip option]
           [--clean option] [-c N] [--ignore-errors] [-v {0,1,2,3,4}]
           DSC script

positional arguments:
  DSC script            DSC script to execute.

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  -o str                Benchmark output. It overwrites "DSC::output" defined
                        in DSC file. (default: None)
  --target str [str ...]
                        This argument can be used in two contexts: 1) When
                        used without "--remove" it specifies "DSC::run" in DSC
                        file. Input should be quoted string(s) defining one or
                        multiple valid DSC pipelines (multiple pipelines
                        should be separated by space). 2) When used along with
                        "--remove" it specifies one or more computational
     

Here we focus on expanding some options that might require extra information than printed on the interface to fully comprehend:

*  `-v` controls verbosity level. Default level is 2 (recommended), which displays necessary runtime info and uses a progressbar to display DSC progress. If you encounter errors you can increase the verbosity level to output more information (eg `-v4`) to diagnose the cause.

* `--target`: this option takes multiple elements.
  * It is possible to overwrite `DSC::run` (a required entry in DSC file) via `--target`. For example, in DSC file the `run` entry that defines a pipeline:
  
      ```
      run: simulate * method * score
      ```
      
      can be overwritten with `--target "simulate * method"` so that the pipeline `simulate * target` will be executed instead. This option is typically useful to execute DSC benchmark bit by bit.
  * When used with `--clean` it specifies the module(s) whose output are to be "clean"-ed up. Therefore only modules or names of ensembles are valid input in this context.
* `--skip`: unlike with `Make` / `Snakemake` that determines whether or not to re-execute based on time stamps, DSC creates HASH for modules that takes into consideration all input parameter, input and output module variables, and the content of the module script if applicable. When all these information agree with a previous execution it will by default skip those runs. This behavior can be changed by this `--skip` option. 
  * `--skip none` will re-execute everything: it will remove and ignore any existing output files. 
  * `--skip all` will not perform any module computations. It will only construct the execution meta-data and create a DSC database that one can query from. That is, `dsc <script> --skip all` followed by `dsc-query <output> -o` will help one understand the expected results from specified DSC benchmark. This can also be used as sanity check when benchmark is only partially completed. If you are uncertain about the scale of the benchmark, for example, it is recommanded to run DSC with `--skip all` and use `dsc-query <output> -o` to view the benchmark structure.
* `-c` configures parallel computing. Since DSC is designed to be executed in either local or remote computers, `-c` configures the number of CPU threads used when computing in local and number of jobs to be sent when computing on the remote computers. Default value is in fact set to using half of the local computers CPU threads. Thus the displayed default `20` CPUs in the documentation above is result of running `dsc -h` on a computer with a total of 40 threads. This number should be specifically configured if one wants to use more (or less) computing power on a desktop; and should definitely be configured for remote job executions.
* `--host` **FIXME: removed since version 0.2.2. Will be fixed after new remote execution model are thoroughly tested.**
* `--ignore-errors` is a flag which, when added, will ignore errors from user provided scripts thus keep the benchmark running. There are often situations when we use other people's software packages that produces errors in random corner cases that will interrupt the benchmark from moving forward. Instead of throwing an error, DSC will **store the problematic chunk of code that resulted in an error to the output, creating a dummy output file that keeps DSC moving**. These dummy output will natually result in a chain reaction of failure in every other module that depends on them. But the benchmark will therefore complete, after which one can check these dummy output for scripts that produced these errors.
* `--distribute`: **FIXME: removed since version 0.2.2 because of unresolved technical difficulties on DSC server's end. Future version may bring back this feature when DSC server is properly configured.** This option will bundle the DSC benchmark into a tarball that can be uploaded to `shinydsc` for data query and visualization, or be transfered to other computational environment. DSC benchmarks should be released with this command. When used without any argument it will pack project meta information, DSC configuration and annotation script (if applicable), and benchmark output into a tarball. This tarball contains enough information to be used with `shinydsc`. To port the complete benchmark, one should specify additional files such as computational scripts and data used, eg, `--distribute /path/to/scripts`. 

## DSC query program

In [2]:
! dsc-query -h

usage: dsc-query [-h] [--version] -o str [--limit N] [--title str]
                 [--description str [str ...]] [-t WHAT [WHAT ...]]
                 [-c WHERE [WHERE ...]] [-g G:A,B [G:A,B ...]] [--add-path]
                 [--language str] [--addon str [str ...]] [-v {0,1,2,3,4}]
                 DSC output folder

positional arguments:
  DSC output folder     Path to DSC output.

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  -o str                Output notebook / data file name. In query
                        applications if file name ends with ".rds" then only
                        data file will be saved as result of query. Otherwise
                        both data file and a notebook that displays the data
                        will be saved. (default: None)
  --limit N             Number of rows to display for tables. Default is to
                        display it for all

This is a companion program to `dsc` that can be used to extract results from benchmark for given conditions. Although `dsc-query` works as is, we are extending it to meet language specific demands by creating separate software packages that wraps this command and enhances -- the approach differs from language to language and will be documented in language specific manner. Therefore we will not provide in-depth documentation to this program.